In [1]:
from sshtunnel import SSHTunnelForwarder
from clickhouse_driver import Client
import pandas as pd
from config import *

In [2]:
repo_ids=[repo["id"] for repo in repos]
repo_names=[repo["name"].split('/')[1] for repo in repos]

In [ ]:
with SSHTunnelForwarder(
    (server_address,server_port),
    ssh_username=server_user_name,
    ssh_password=server_user_password,
    remote_bind_address=(remote_address,remote_port)) as server:
    client = Client(host='127.0.0.1',port=server.local_bind_port,user=remote_user_name,
        password=remote_user_passward,
        database=database)
    table='year2020'
    for i in range(len(repos)):
        repo_id=repo_ids[i]
        pc=f'''
            SELECT toDate(created_at) AS datetime,COUNT(distinct actor_id) AS pc FROM {table} WHERE repo_id={repo_id} GROUP BY datetime
        '''
        wc=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS wc FROM {table} WHERE type='WatchEvent' AND repo_id={repo_id} GROUP BY datetime
        '''
        fc=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS fc FROM {table} WHERE type='ForkEvent' AND repo_id={repo_id} GROUP BY datetime
        '''
        icc=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS icc FROM {table} WHERE type='IssueCommentEvent' AND action='created' AND repo_id={repo_id} GROUP BY datetime
        '''
        oic=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS oic FROM {table} WHERE type='IssuesEvent' AND action='opened' AND repo_id={repo_id} GROUP BY datetime
        '''
        cic=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS cic FROM {table} WHERE type='IssuesEvent' AND action='closed' AND repo_id={repo_id} GROUP BY datetime
        '''
        opc=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS opc FROM {table} WHERE type='PullRequestEvent' AND action='opened' AND repo_id={repo_id} GROUP BY datetime
        '''
        cpdc=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS cpdc FROM {table} WHERE type='PullRequestEvent' AND action='closed' AND pull_merged!=1 AND repo_id={repo_id} GROUP BY datetime
        '''
        cpc=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS cpc,SUM(pull_commits) AS commits,SUM(pull_additions) AS additions,SUM(pull_deletions) AS deletions FROM {table} WHERE type='PullRequestEvent' AND action='closed' AND pull_merged=1 AND repo_id={repo_id} GROUP BY datetime
        '''
        prc=f'''
            SELECT toDate(created_at) AS datetime,COUNT(*) AS prc FROM {table} WHERE type='PullRequestReviewCommentEvent' AND action='created' AND repo_id={repo_id} GROUP BY datetime
        '''
        sql=f'''
            SELECT pc.datetime AS datetime,pc AS participate,wc AS Star,fc AS Fork,
            icc AS IssueComment,oic AS OpenIssue,cic AS ClosedIssue,
            opc AS PROpen,cpdc AS PRdeclined,
            cpc AS PRMerged,commits,additions AS CodeAddedLines,deletions AS CodeDeletedLines,prc AS PRReviewComment
            FROM ({pc}) AS pc
            LEFT JOIN ({wc}) AS wc
            ON pc.datetime=wc.datetime
            LEFT JOIN ({fc}) AS fc
            ON pc.datetime=fc.datetime
            LEFT JOIN ({icc}) AS icc
            ON pc.datetime=icc.datetime
            LEFT JOIN ({oic}) AS oic
            ON pc.datetime=oic.datetime
            LEFT JOIN ({cic}) AS cic
            ON pc.datetime=cic.datetime
            LEFT JOIN ({opc}) AS opc
            ON pc.datetime=opc.datetime
            LEFT JOIN ({cpdc}) AS cpdc
            ON pc.datetime=cpdc.datetime
            LEFT JOIN ({cpc}) AS cpc
            ON pc.datetime=cpc.datetime
            LEFT JOIN ({prc}) AS prc
            ON pc.datetime=prc.datetime
        '''
        df=client.query_dataframe(sql)
        repo_name=repo_names[i]
        df.to_csv('data/'+repo_name+'.csv',index=None)
        